In [1]:

from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
import time
import codecs
import pandas as pd
import xml.etree.ElementTree as ET
import pymarc
from pymarc import Record,Field
import marcx
import io
from lxml import etree
import json
import glob
#from spst import SPST
import os
os.chdir('/Users/jammerman/Google Drive/git/hathitrust')
import string
#os.listdir('.')

In [2]:
def get_oclc(rec):
    _035s = rec.findall('*/[@tag="035"]')
    for _035 in _035s:

        try:
            _035a = _035.find('*/[@code="a"]').text
            if '(OCoLC)' in _035a:
                return(_035a)
            if _035a.isdigit():
                #print(_035a)
                return(_035a)
            #else:
            #    print(_035a)
        except Exception as e:
            
            pass    
    return(False)
    
def get_mtype(rec):
    leader = rec.find('./leader').text
    mtype = leader[6:8]
    return(mtype)

def get_mms_id(rec):
    mms_id = rec.find('*/[@tag="001"]').text
    return(mms_id)

def get_issn(rec):
    _022s = rec.findall('*/[@tag="022"]')
    for _022 in _022s:
        try:
            _022a = _022.find('*/[@code="a"]').text
            if '-' in _022a:
                return(_022a)
            if _022a.isdigit():
                return(_022a)
        except Exception as e:
            pass    
    return('')

def get_govdoc(rec):
    try:
        _008 = rec.find('*/[@tag="008"]').text
        if _008[28] == 'f':
            return('1')
    except Exception as e:
        pass
    return('0')

def get_holding_status(_949):
    try:
        _949g = _949.find('*/[@code="g"]').text
        #print(_949g)
        if 'BU' in _949g: ## in the BU Legacy Library
            #print('Withdrawn')
            return('WD')
    except Exception as e:
        pass   
    try:
        _949f = _949.find('*/[@code="f"]').text ## Missing or Lost status
        if 'MISSING' in _949f:
            #print('missing')
            return('LM')
        if 'LOST_LOAN' in _949f:
            #print('lost')
            return('LM')
    except Exception as e:
        #print(ET.dump(_949))
        #print()
        pass
    return''
def get_ser_fields(rec):
    ser_string = ''.join(get_oclc(rec)) + '\t'
    ser_string += get_mms_id(rec) + '\t'
    ser_string += get_issn(rec) + '\t'
    ser_string += get_govdoc((rec)) + '\n'
    return(ser_string)

def get_spm_fields(rec):
    _949s = rec.findall('*/[@tag="949"]')
    spm_string = ''
    for _949 in _949s:
        spm_string += get_oclc(rec) + '\t'
        spm_string += get_mms_id(rec) + '\t'
        spm_string += get_holding_status(_949) + '\t'
        spm_string += '\t' ## Condition not reportable
        spm_string += get_govdoc(rec) + '\n'
    return(spm_string)

def get_mpm_fields(rec):
    mpm_string = ''
    mpm_string += get_oclc(rec) + '\t'
    mpm_string += get_mms_id(rec) + '\t'
    mpm_string += get_holding_status(rec) + '\t'
    mpm_string += '\t' ## Condition not reportable
    _949s = rec.findall('*/[@tag="949"]')
    vols = []
    for _949 in _949s:
        try:
            _949c = _949.find('*/[@code="c"]').text
            vols.append(_949c)
        except Exception as e:
            pass  
    vols.sort()
    enum = ''.join(str(e) +'; ' for e in vols)
    mpm_string += enum +'\t' 
    mpm_string += get_govdoc(rec) + '\n'
    return(mpm_string)

def is_mpm(rec):
    _300s = rec.findall('*/[@tag="300"]')
    for _300 in _300s:
        try:
            _300a = _300.find('*/[@code="a"]').text
            if 'volumes' in _300a:
                return(True)
        except Exception as e:
            pass
        try:
            if 'v.' in _300a:
                return(True)
        except Exception as e:
            pass   

    _852s = rec.findall('*/[@tag="852"]')
    for _852 in _852s:
        try:
            _852i = _852.find('*/[@code="i"]').text
            if ' v. ' in _852i:
                return(True)
        except Exception as e:
            pass   

    _949s = rec.findall('*/[@tag="949"]')
    for _949 in _949s:
        try:
            _949a = _949.find('*/[@code="a"]').text
            if 'v.' in _949a:
                return(True)
        except Exception as e:
            pass
        try:
            _949e = _949.find('*/[@code="e"]').text
            if 'v.' in _949e:
                return(True)
        except Exception as e:
            pass   
    return(False)

In [3]:
file_list = glob.glob('2016/ht_*.xml')
ser_out = io.open('bu_serials_20160629.tsv','w')
spm_out = io.open('bu_single-part_20160629.tsv','w')
mpm_out = io.open('bu_multi-part_20160629.tsv','w')
err_out = io.open('bu_err_20160629.tsv','w')
mtypes = {}
fcount = 0
ser = 0
spm = 0
mpm = 0
acceptable = {'aa','ab','ac','am','ai','as','ca','cb','cc','cm','ci','cs','ea','eb','ec','em','ei','es','tm'}
for f in file_list[0:]:
    print(str(fcount),f)
    fcount += 1
    tree = ET.parse(f)
    collection = tree.getroot()
    records = collection.findall('./record')
    for record in records[0:]:
        #print(ET.tostring(record))
        oclc_num = get_oclc(record)
        mms_id = get_mms_id(record)
        #print(oclc_num,mms_id)
        if oclc_num != False:
            mtype = get_mtype(record)
            if mtype in mtypes.keys():
                mtypes[mtype] += 1
            else:
                mtypes[mtype] = 1
            if mtype in acceptable:
                if mtype == 'as' or mtype == 'ai' or mtype == 'cs' or mtype == 'ci' or mtype == 'es' or mtype == 'ei': ## we have a serial
                    #print(get_mms_id(record))
                    fields = get_ser_fields(record)
                    ser_out.write(fields)
                    ser += 1
                    #print(fields)
                #if mtype != 'as': ## we have a monograph
                else:
                    if is_mpm(record) == True:
                        fields = get_mpm_fields(record)
                        mpm_out.write(fields)
                        mpm += 1
                    else:
                        fields = get_spm_fields(record)
                        spm_out.write(fields)
                        spm += 1
            else:
                mms_id = get_mms_id(record)
                err_out.write(mms_id + '\t' + oclc_num + '\n')
            #if oclc_num.isdigit():
            #    print(oclc_num,mms_id)
        else:
            #pass
            print(oclc_num,mms_id)
ser_out.close()
spm_out.close()
mpm_out.close()
err_out.close()

0 2016/ht_pt_2016070614_20695072230001161_new_1.xml
False 99174707640001161
False 99175111580001161
False 99175237210001161
False 99175261090001161
False 99175388620001161
False 99175385110001161
False 99175283020001161
False 99175394900001161
False 99175524130001161
False 99175534940001161
False 99175560640001161
False 99175575670001161
False 99175408670001161
False 99175420410001161
False 99175418030001161
False 99175413030001161
False 99175421830001161
False 99175446380001161
False 99175635490001161
False 99175772480001161
False 99175744690001161
False 99175761610001161
False 99175782170001161
False 99175782820001161
False 99175792020001161
False 99175868080001161
False 99175898980001161
False 99175828250001161
False 99175878100001161
False 99175918460001161
False 99175934210001161
False 99175947720001161
False 99176070590001161
False 99176066940001161
False 99176160540001161
False 99176176430001161
False 99176195240001161
False 99176195440001161
False 99176186510001161
False 991761

In [4]:
print(ser)
print(mpm)
print(spm)

33991
71949
1097348


In [5]:
for k,v in mtypes.items():
    print(k,v)

m  3
ec 3
ea 1
m8 8
gm 1249
ja 2
aI 1
s  2
dc 1
ib 1
ra 2
im 2200
hs 4
ai 976
a1 5
kc 12
km 95
rc 1
mI 9
c0 1
cs 8
j  1
bc 132
hm 89
a  1
c2 1
gc 2
mm 123
pa 1
tm 11084
a0 3
cm 46831
 a 1
pm 1
nc 3
a2 5
tc 4
l  1
ia 4
em 392
ng 4
45 1
5m 1
mi 1
c  1
mK 1
15 1
a4 1
4c 1
dn 1
na 28
ms 22
cI 1
ad 6
bm 58
aa 363
a9 1
 n 3
   4
 u 2
cc 14
pc 11
jc 2
 d 5
mc 1
ac 150
3m 1
jm 30072
ic 3
un 2
qm 1
16 1
I  1
a8 1
om 44
a5 2
as 33007
g  3
0m 2
js 2
a6 4
ca 6372
 m 8
a3 5
sI 1
0c 1
 c 4
am 1104087
6c 1
rm 51
dd 1
nm 1
dm 299
gs 4
